In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 30/117

# KÜTÜPHANELER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
data['egitim'].value_counts()

In [ ]:
data['is_durumu'].value_counts()

In [ ]:
data['meslek_grubu'].value_counts()

# Monthly ve train datalarının okunması- derlenmesi ve birleştirilmesi 

In [ ]:
data_train=pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/train.csv")
data_month=pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/monthly_expenditures.csv")

In [ ]:
data_train.info()

In [ ]:
y=data_train['target']
x=data_train[['yas','kidem_suresi','egitim','is_durumu','meslek_grubu']]

In [ ]:
x.head()


In [ ]:

x=pd.get_dummies(x,columns=["egitim","is_durumu","meslek_grubu"])
x.info()

In [ ]:
data_train.info()
data_month.info()

In [ ]:
data_month.head()

In [ ]:
#train datada meslek grubunun 0.36 sı null olduğu için datadan silindi
data_train=data_train.drop(['meslek_grubu'],axis=1)
data_train.info()

In [ ]:
data_train=data_train.drop(['meslek_grubu','egitim'],axis=1)
data_train.info()

In [ ]:
#train datada geri kalan eksik gözlemleri siliyoruz
data_train=data_train.dropna()
data_train.info()

* Monthly tablosunda denenen senaryolar

In [ ]:
exp= data_month.groupby(['musteri', 'sektor'])['aylik_toplam_tutar'].sum().unstack().fillna(1)
exp.head()

In [ ]:
#monthly datasının düzenlenmesi
exp= data_month.groupby(['musteri', 'sektor'])['aylik_toplam_tutar'].sum().unstack().fillna(0)

In [ ]:
exp= data_month.groupby(['musteri', 'sektor'])['islem_adedi'].sum().unstack().fillna(0)

In [ ]:
exp= data_month.groupby(['musteri', 'tarih'])['aylik_toplam_tutar'].sum().unstack().fillna(0)

In [ ]:
data_month.head()

In [ ]:
exp.head()

In [ ]:
data_train=data_train.set_index('musteri')
data_train.head()

In [ ]:
joined=data_train.join(exp)
joined.head()

In [ ]:
joined.info()

In [ ]:
#dummy işlemi mesleksiz ve eğitimsiz
joined_dum=pd.get_dummies(joined,columns=["is_durumu"])
joined_dum.info()

In [ ]:
#dummy işlemi mesleksiz
joined_dum=pd.get_dummies(joined,columns=["egitim","is_durumu"])
joined_dum.info()

In [ ]:
#dummy işlemi meslekli
joined_dum=pd.get_dummies(joined,columns=["egitim","is_durumu","meslek_grubu"])
joined_dum.info()

In [ ]:
joined_dum.head()

In [ ]:
##Bu kategoriler test datasında olmadığı için siliyoruz is_durumu_3773727d6e ,egitim_2eb5ddd72c
y=joined["target"]
x=joined_dum.drop(["tarih","target","egitim_2eb5ddd72c","is_durumu_3773727d6e"],axis=1)


In [ ]:
x.info()

In [ ]:
y.head()

In [ ]:
x.info()

* Veri seti imblance olduğu için algoritmalarda SMOTE, UNDERSAMPLING, OVERSAMPLING yöntemleri tek tek denendi en iyi model accuracy score Oversampling uygulanan veri seti ile bulundu.

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE
smote= SMOTE(random_state=12)
X_resampled_smote, y_resampled_smote = smote.fit_resample(x, y)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=123)
X_resampled_under, y_resampled_under = ros.fit_resample(x, y)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train,X_test,y_train,y_test=train_test_split(x, y,test_size=0.2, random_state=42)
print("X_train", X_train.shape)

print("y_train",y_train.shape)

print("X_test",X_test.shape)

print("y_test",y_test.shape)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros =RandomOverSampler(random_state=123)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros =RandomOverSampler(random_state=123)
X_resampled, y_resampled = ros.fit_resample(x, y)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train_smote,X_test_smote,y_train_smote,y_test_smote=train_test_split(X_resampled_smote, y_resampled_smote,test_size=0.1, random_state=42)
print("X_train", X_train_smote.shape)

print("y_train",y_train_smote.shape)

print("X_test",X_test_smote.shape)

print("y_test",y_test_smote.shape)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train_under,X_test_under,y_train_under,y_test_under=train_test_split(X_resampled_under, y_resampled_under,test_size=0.1, random_state=42)
print("X_train", X_train_under.shape)

print("y_train",y_train_under.shape)

print("X_test",X_test_under.shape)

print("y_test",y_test_under.shape)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train,X_test,y_train,y_test=train_test_split(X_resampled, y_resampled,test_size=0.1, random_state=42)
print("X_train", X_train.shape)

print("y_train",y_train.shape)

print("X_test",X_test.shape)

print("y_test",y_test.shape)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()


# Datanın test-train ayrımı

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.1, random_state=42)
print("X_train", X_train.shape)

print("y_train",y_train.shape)

print("X_test",X_test.shape)

print("y_test",y_test.shape)

In [ ]:
y.value_counts()

y_train.value_counts()

y_test.value_counts()


# Algoritmalar

# CATBOOST

In [ ]:
model = CatBoostClassifier().fit(X_train, y_train)
y_pred_model = model.predict(X_test)
print(classification_report(y_test,y_pred_model ))


In [ ]:
cross_val_score(model,X_test,y_test,cv=10).mean()

# GBM

In [ ]:
gbm_model = GradientBoostingClassifier().fit(X_train, y_train)
y_pred_gbm = gbm_model.predict(X_test)
print(classification_report(y_test,y_pred_gbm ))
cross_val_score(gbm_model,X_test,y_test,cv=10).mean()

In [ ]:
gbm_model_tune = GradientBoostingClassifier(random_state=42).fit(X_resampled, y_resampled)
y_pred_gbm = gbm_model_tune.predict(X_test)
print(classification_report(y_test,y_pred_gbm ))


In [ ]:
cross_val_score(gbm_model_tune,X_test,y_test,cv=10).mean()

In [ ]:
?gbm_model

In [ ]:
gbm_model_samp = GradientBoostingClassifier(random_state=12).fit(X_train_under, y_train_under)
y_pred_gbm_samp = gbm_model_samp.predict(X_test_under)
print(classification_report(y_test_under,y_pred_gbm_samp ))
cross_val_score(gbm_model_samp,X_test_under,y_test_under,cv=10).mean()

In [ ]:
confusion_matrix(y_test,y_pred_gbm)


# KNN

In [ ]:
knn = KNeighborsClassifier().fit(X_train_under, y_train_under)
y_pred_knn = knn.predict(X_test_under)
print(classification_report(y_test_under,y_pred_knn))
cross_val_score(knn,X_test_under,y_test_under,cv=10).mean()

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
print(classification_report(y_test,y_pred_knn))
cross_val_score(knn_model,X_test,y_test,cv=10).mean()

In [ ]:
confusion_matrix(y_test,y_pred_knn)


# LOJİSTİK REGRESYON

In [ ]:
loj=LogisticRegression()
loj_model=loj.fit(X_train, y_train)
y_pred_mod = loj_model.predict(X_test)
print(classification_report(y_test,y_pred_mod))
cross_val_score(loj_model,X_test,y_test,cv=10).mean()

# NAİVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
print(classification_report(y_test,y_pred_nb))
cross_val_score(nb_model,X_test,y_test,cv=10).mean()


In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train_under, y_train_under)
y_pred_nb = nb_model.predict(X_test_under)
print(classification_report(y_test_under,y_pred_nb))
cross_val_score(nb_model,X_test_under,y_test_under,cv=10).mean()


In [ ]:
confusion_matrix(y_test,y_pred_nb)


# Neural Network

In [ ]:
nn_model_meslekvar = MLPClassifier( random_state=42).fit(X_train, y_train)
y_pred_nn= nn_model_meslekvar.predict(X_test)
print(classification_report(y_test,y_pred_nn))
cross_val_score(nn_model_meslekvar,X_test,y_test,cv=10).mean()

In [ ]:
confusion_matrix(y_test,y_pred_nn)

In [ ]:
nn_model = MLPClassifier( random_state=42).fit(X_resampled, y_resampled)
y_pred_nn= nn_model.predict(X_test)
print(classification_report(y_test,y_pred_nn))
cross_val_score(nn_model,X_test,y_test,cv=10).mean()

In [ ]:
confusion_matrix(y_test_under,y_pred_nn)

# LGBM

In [ ]:
lgbm_samp=LGBMClassifier().fit(X_train_under, y_train_under)
y_pred_= lgbm_samp.predict(X_test_under)
print(classification_report(y_test_under,y_pred_))
cross_val_score(lgbm_samp,X_test_under,y_test_under,cv=10).mean()

In [ ]:
confusion_matrix(y_test_under,y_pred_)

In [ ]:

Lgbm_params = {"learning_rate" : [0.001, 0.01, 0.1, 0.05],
             "n_estimators": [100,500,100],
             "max_depth": [3,5,10],
             "min_samples_split": [2,5,10]}

In [ ]:
lgbm=LGBMClassifier(n_estimators=300,learning_rate=0.1,random_state=42).fit(X_train, y_train)
y_pred_train= lgbm.predict(X_test)
print(classification_report(y_test,y_pred_train))
cross_val_score(lgbm,X_test,y_test,cv=10).mean()

In [ ]:
y_pred_train= lgbm.predict(x )
print(classification_report(y,y_pred_train))
cross_val_score(lgbm,x,y,cv=10).mean()

In [ ]:
y_pred_train= lgbm.predict(X_resampled )
print(classification_report(y_resampled,y_pred_train))
cross_val_score(lgbm,X_resampled,y_resampled,cv=10).mean()

In [ ]:
lgbm_train=LGBMClassifier(n_estimators=200,learning_rate=0.02,random_state=42).fit(X_train, y_train)
y_pred_train= lgbm_train.predict(X_test)
print(classification_report(y_test,y_pred_train))
cross_val_score(lgbm_train,X_test,y_test,cv=10).mean()

In [ ]:
y_pred_train= lgbm_train.predict(X_resampled )
print(classification_report(y_resampled,y_pred_train))
cross_val_score(lgbm_train,X_resampled,y_resampled,cv=10).mean()

In [ ]:
#0.1 test size
lgbm_=LGBMClassifier().fit(X_train, y_train)
y_pred_= lgbm_.predict(X_test)
print(classification_report(y_test,y_pred_))
cross_val_score(lgbm_,X_test,y_test,cv=10).mean()


In [ ]:
y_pred_= lgbm_.predict(x)
print(classification_report(y,y_pred_))
cross_val_score(lgbm,x,y,cv=10).mean()

In [ ]:
confusion_matrix(y_test,y_pred_)

# DECISION TREE

In [ ]:
dt = DecisionTreeClassifier(random_state=42,max_depth=30,min_samples_split=2)
dt_model = dt.fit(X_train, y_train)
y_pred_dt= dt_model.predict(X_test)
print(classification_report(y_test,y_pred_dt))
cross_val_score(dt_model,X_test,y_test,cv=10).mean()

In [ ]:
confusion_matrix(y_test,y_pred_rf)

# XGB

In [ ]:
xgb_model = XGBClassifier().fit(X_train, y_train)
y_pred_xgb= xgb_model.predict(X_test)
print(classification_report(y_test,y_pred_xgb))
cross_val_score(xgb_model,X_test,y_test,cv=10).mean()

In [ ]:
y_pred_train= xgb_model.predict(X_resampled )
print(classification_report(y_resampled,y_pred_train))
cross_val_score(xgb_model,X_resampled,y_resampled,cv=10).mean()

In [ ]:
y_pred_train= xgb_model.predict(x)
print(classification_report(y,y_pred_train))
cross_val_score(xgb_model,x,y,cv=10).mean()

In [ ]:

xgb_model_samp = XGBClassifier().fit(X_resampled, y_resampled)
y_pred_xgb_samp= xgb_model_samp.predict(X_test)
print(classification_report(y_test,y_pred_xgb))
cross_val_score(xgb_model_samp,X_test,y_test,cv=10).mean()

In [ ]:
y_pred_train= xgb_model_samp.predict(x)
print(classification_report(y,y_pred_train))
cross_val_score(xgb_model_samp,x,y,cv=10).mean()

In [ ]:
confusion_matrix(y_test_under,y_pred_xgb)

# Modellerin Test Datası İçin Tahmini

In [ ]:
test=pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/test.csv")
test.head()

In [ ]:
exp.head()

In [ ]:
test=test.set_index('musteri')
test.head()

In [ ]:
joined_test=test.join(exp)
joined_test.head()

In [ ]:
test=pd.get_dummies(test,columns=["egitim","is_durumu","meslek_grubu"])
test.info()

In [ ]:
test=test.drop(['tarih'],axis=1)
test.info()

In [ ]:
dum_test=pd.get_dummies(test,columns=["is_durumu","meslek_grubu","egitim"])
dum_test.info()

In [ ]:
joined_dum_test=pd.get_dummies(joined_test,columns=["egitim","is_durumu","meslek_grubu"])
joined_dum_test.info()

In [ ]:
joined_dum_test=joined_dum_test.drop(['meslek_grubu','tarih','is_durumu'],axis=1)
joined_dum_test.info()

In [ ]:
dum_test=dum_test.drop(['tarih'],axis=1)
dum_test.info()

In [ ]:
joined_dum_test.info()

In [ ]:
joined_dum_test=joined_dum_test.drop(['tarih'],axis=1)
joined_dum_test.info()

In [ ]:
target=gbm_model_tune.predict(dum_test)
target[0:20]

In [ ]:
target= knn_model.predict(dum_test)
target[0:20]

In [ ]:
target= lgbm.predict(joined_dum_test)
target[0:20]

In [ ]:
target2= gbm_model_tune.predict(joined_dum_test)
target2[0:20]

In [ ]:
target_sub=pd.DataFrame(target,columns=['target'])

In [ ]:
submit=dum_test.index

In [ ]:
submit_sub=pd.DataFrame(submit,columns=['musteri'])

In [ ]:
submit_sub.head()

In [ ]:
submit_son=pd.concat([submit_sub,target_sub],axis=1)

In [ ]:
submit_son.head()

In [ ]:
submit_son.to_csv("submit_lgbm.csv",index = False)

In [ ]:
ornek=pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/sample_submission.csv")
ornek.head()